<a href="https://colab.research.google.com/github/martinbedia/mineria/blob/main/ejercicio%202.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Ejercicio 2
Crea un nuevo programa para procesar el nuevo formato de ticket para obtener un archivo csv en el que se muestre el ID del ticket, producto y precio. Genera también un segundo csv donde para cada ticket se muestre la dirección del establecimiento, la hora de compra, el total del ticket, el descuento generado y el acumulado.

In [32]:
!pip install PyPDF2

In [33]:
import pandas as pd
import numpy as np
import glob, os
import PyPDF2
import re
import csv
from datetime import datetime

Antes de comenzar a analizar los tickets debo cargarlos en el entorno de Google Collab y asegurarme que los tengo en el panel de la izquierda.

## Funciones individuales
Seguiremos el mismo procedimiento que el ejercicio 1, elaboraremos pequeñas funciones individuales para cada dato que pide el enunciado, para posteriormente unirlas en la función final. De esta manera voy viendo que me van saliendo los datos que necesito.

In [106]:
# leer archivos de un tipo concreto de una carpeta y sus subcarpetas
# recursive para que busque en las subcarpetas
ruta_carpeta_raiz = '.'
tickets = glob.glob(os.path.join(ruta_carpeta_raiz, '**/*.pdf'),recursive=True)
tickets

['./Ticket-13_11_2023-2.pdf',
 './Ticket-23_11_2023-2.pdf',
 './Ticket-3_11_2023-2.pdf']

In [35]:
# leer el ticket
pdf_reader = PyPDF2.PdfReader(tickets[0])    # conectar el archivo pdf con el codigo en python
ticket = pdf_reader.pages[0]   # acceder a la primera pagina
texto_ticket = ticket.extract_text()
texto_ticket

'2LIMERKA, S.A.U. CIF A33093097 \nCASTIELLO, 145 - LUGO DE LLANERA \nCompra realizada en nuestro supermercado \nC/ Muñoz Degraíin, 5, Oviedo \n  ENSALADA RUCULA IFA 0,72 C \nEOG.CER-RUS.BAKERY 2,67 R \nTOTAL 3,39 EUR \nTARJETA 3,35 EUR \nTipo Base IVA \nR 10,00% 2,43 0,24 \nc 0,00% 0,72 0,00 \nTotal Fra. Simplif. 3,39 \n  \nSaldo próximo Vale Mensual:0,37 EUR. \nEuros conseguidos:0,03 EUR. \nNúmero Tarjeta: 112000005970€985 \ne de de de de de de de de de e de de de de e de de de de de de de de e de de de de de de de de de de A de e de \n  \nEMPRESA: 00000001 CENTRO-0195 TPV:0002 \nOPERAD.- 00000001 NO.OPERACION: 085332 \nFECHA -: 13/11/2023 HORA: 19:57:52 \nMOBILE EMV /¿AUTORIZACION: 502024 \nTARJETA : ed \nAUTENTICACION: FIRMA NO NECESARIA \nAID: A0000000041010 \nMASTERCARD \nATC: 0455 ARC: 00 \nMASTERCARD CONTACTLESS \nREFERENCIA: 459180 SESION:13112023-001 \nde de de de de de de de de de e de V E N T a de de de de de de de de de de de de de \nTOTAL: 3.39 EUR \nde de de de de de de d

In [36]:
# texto del ticket por lineas
lineas = np.array(texto_ticket.split('\n'))
lineas

array(['2LIMERKA, S.A.U. CIF A33093097 ',
       'CASTIELLO, 145 - LUGO DE LLANERA ',
       'Compra realizada en nuestro supermercado ',
       'C/ Muñoz Degraíin, 5, Oviedo ', '  ENSALADA RUCULA IFA 0,72 C ',
       'EOG.CER-RUS.BAKERY 2,67 R ', 'TOTAL 3,39 EUR ',
       'TARJETA 3,35 EUR ', 'Tipo Base IVA ', 'R 10,00% 2,43 0,24 ',
       'c 0,00% 0,72 0,00 ', 'Total Fra. Simplif. 3,39 ', '  ',
       'Saldo próximo Vale Mensual:0,37 EUR. ',
       'Euros conseguidos:0,03 EUR. ',
       'Número Tarjeta: 112000005970€985 ',
       'e de de de de de de de de de e de de de de e de de de de de de de de e de de de de de de de de de de A de e de ',
       '  ', 'EMPRESA: 00000001 CENTRO-0195 TPV:0002 ',
       'OPERAD.- 00000001 NO.OPERACION: 085332 ',
       'FECHA -: 13/11/2023 HORA: 19:57:52 ',
       'MOBILE EMV /¿AUTORIZACION: 502024 ', 'TARJETA : ed ',
       'AUTENTICACION: FIRMA NO NECESARIA ', 'AID: A0000000041010 ',
       'MASTERCARD ', 'ATC: 0455 ARC: 00 ', 'MASTERCARD CONTACTL

In [37]:
def leer_lineas_de_pdf (nombre_del_pdf):
  pdf_reader = PyPDF2.PdfReader(nombre_del_pdf)
  ticket = pdf_reader.pages[0]
  texto_ticket = ticket.extract_text()
  lineas = np.array(texto_ticket.split('\n'))
  return lineas

In [38]:
lineas_ticekt1 = leer_lineas_de_pdf('/content/Ticket-13_11_2023-2.pdf')
lineas_ticekt1

array(['2LIMERKA, S.A.U. CIF A33093097 ',
       'CASTIELLO, 145 - LUGO DE LLANERA ',
       'Compra realizada en nuestro supermercado ',
       'C/ Muñoz Degraíin, 5, Oviedo ', '  ENSALADA RUCULA IFA 0,72 C ',
       'EOG.CER-RUS.BAKERY 2,67 R ', 'TOTAL 3,39 EUR ',
       'TARJETA 3,35 EUR ', 'Tipo Base IVA ', 'R 10,00% 2,43 0,24 ',
       'c 0,00% 0,72 0,00 ', 'Total Fra. Simplif. 3,39 ', '  ',
       'Saldo próximo Vale Mensual:0,37 EUR. ',
       'Euros conseguidos:0,03 EUR. ',
       'Número Tarjeta: 112000005970€985 ',
       'e de de de de de de de de de e de de de de e de de de de de de de de e de de de de de de de de de de A de e de ',
       '  ', 'EMPRESA: 00000001 CENTRO-0195 TPV:0002 ',
       'OPERAD.- 00000001 NO.OPERACION: 085332 ',
       'FECHA -: 13/11/2023 HORA: 19:57:52 ',
       'MOBILE EMV /¿AUTORIZACION: 502024 ', 'TARJETA : ed ',
       'AUTENTICACION: FIRMA NO NECESARIA ', 'AID: A0000000041010 ',
       'MASTERCARD ', 'ATC: 0455 ARC: 00 ', 'MASTERCARD CONTACTL

In [39]:
def extraer_id_ticket(lineas):
    id_ticket_match = re.search(r'\d*-\d*-\d*', '\n'.join(lineas))
    id_ticket = id_ticket_match.group() if id_ticket_match else None

    return id_ticket

extraer_id_ticket(lineas_ticekt1)

'0195-02-23176767'

In [40]:
def extraer_productos(lineas):
  productos = []
  for linea in lineas:
        producto_match = re.search(r'[A-ZÑñÁÉÍÓÚáéíóú][A-ZÑñÁÉÍÓÚáéíóú/0-9\s]+((\d{1,2},\d{1,2}\s*){1,2})', linea, flags=re.IGNORECASE)
        if producto_match:
            productos.append(producto_match.group())
  return productos

productos_ticket = extraer_productos(lineas_ticekt1)
productos_ticket


['ENSALADA RUCULA IFA 0,72 ',
 'BAKERY 2,67 ',
 'TOTAL 3,39 ',
 'TARJETA 3,35 ',
 'R 10,00',
 'c 0,00']

In [41]:
def obtener_nombre_producto(linea):
    # Buscar coincidencias en la línea
    coincidencia = re.search(r'[A-ZÑñÁÉÍÓÚáéíóú/0-9\s]+[A-ZÑñÁÉÍÓÚáéíóú]+\s', linea, flags=re.IGNORECASE)

    if coincidencia:
        return coincidencia.group().strip()
    else:
        return None

# Obtener los nombres de los productos
nombres_productos = [obtener_nombre_producto(linea) for linea in productos_ticket]

print(nombres_productos)

['ENSALADA RUCULA IFA', 'BAKERY', 'TOTAL', 'TARJETA', None, None]


In [54]:
def obtener_precio_producto(linea):
    # Buscar coincidencias en la línea
    coincidencia = re.search(r'(\d{1,2},\d{1,2})', linea)

    if coincidencia:
        return coincidencia.group()
    else:
        return None

# Obtener los precios de los productos
precios_productos = [obtener_precio_producto(linea) for linea in productos_ticket]

print(precios_productos)

['0,72', '2,67', '3,39', '3,35', '10,00', '0,00']


En la siguiente función obtendré todos los datos que me pide la primera parte del enunciado, es decir, ticket id, nombre del producto y precio.

In [67]:
# todo en la misma funcion
ruta_carpeta_raiz = '.'
tickets = glob.glob(os.path.join(ruta_carpeta_raiz, '**/*.pdf'),recursive=True)
tickets

def ejercicio2(tickets):
  # leer el ticket
  pdf_reader = PyPDF2.PdfReader(tickets)    # conectar el archivo pdf con el codigo en python
  ticket = pdf_reader.pages[0]   # acceder a la primera pagina
  texto_ticket = ticket.extract_text()
  # texto del ticket por lineas
  lineas = np.array(texto_ticket.split('\n'))

  id_ticket_match = re.search(r'\d*-\d*-\d*', '\n'.join(lineas))
  id_ticket = id_ticket_match.group() if id_ticket_match else None

  productos = []
  for linea in lineas:
        producto_match = re.search(r'[A-ZÑñÁÉÍÓÚáéíóú][A-ZÑñÁÉÍÓÚáéíóú/0-9\s]+((\d{1,2},\d{1,2}\s*){1,2})', linea, flags=re.IGNORECASE)
        if producto_match:
            productos.append(producto_match.group())


  # Buscar coincidencias en el nombre
  nombres = []
  for linea in productos:
    nombres_matches = re.search(r'[A-ZÑñÁÉÍÓÚáéíóú/0-9\s]+[A-ZÑñÁÉÍÓÚáéíóú]+\s', linea, flags=re.IGNORECASE)
    nombres.append(nombres_matches.group().strip() if nombres_matches else None)

  # Buscar coincidencias en el precio
  precios = []
  for linea in productos:
    precio_matches = re.search(r'(\d{1,2},\d{1,2})', linea)
    precios.append(precio_matches.group() if precio_matches else None)

  df = pd.DataFrame({
    'id ticket': id_ticket,
    'producto': nombres,
    'precio_individual': precios
  })

  df = df.dropna()
  df = df.loc[df['producto'] != 'TOTAL']
  df = df.loc[df['producto'] != 'TARJETA']


  return df

In [68]:
ejercicio2('/content/Ticket-13_11_2023-2.pdf')

,id ticket,producto,precio_individual
0,0195-02-23176767,ENSALADA RUCULA IFA,"0,72"
1,0195-02-23176767,BAKERY,"2,67"


In [69]:
ejercicio2('/content/Ticket-23_11_2023-2.pdf')

,id ticket,producto,precio_individual
0,0195-02-23182951,SOLOMILLO POLLO POZO,"5,35"


In [70]:
ejercicio2('/content/Ticket-3_11_2023-2.pdf')

,id ticket,producto,precio_individual
0,0195-02-23171045,EUMMUS ENS2ALANDIA,"1,25"
1,0195-02-23171045,PICOS IFA CAMPER,"0,70"


In [71]:
def extraer_descuento(lineas):
    descuento_gen_matches = re.search(r'Euros conseguidos:\d{1,2},\d{1,2}', '\n'.join(lineas))
    descuento_gen = descuento_gen_matches.group() if descuento_gen_matches else None
    descuento_cifra_match = re.search(r'\d{1,2},\d{1,2}', descuento_gen)
    descuento_cifra = descuento_cifra_match.group() if descuento_cifra_match else None

    return descuento_cifra

extraer_descuento(lineas_ticekt1)

'0,03'

In [72]:
# funcion de los descuentos
def ejercicio2_descuento(tickets):
  pdf_reader = PyPDF2.PdfReader(tickets)
  ticket = pdf_reader.pages[0]
  texto_ticket = ticket.extract_text()
  # texto del ticket por lineas
  lineas = np.array(texto_ticket.split('\n'))

  id_ticket_match = re.search(r'\d*-\d*-\d*', '\n'.join(lineas))
  id_ticket = id_ticket_match.group() if id_ticket_match else None

  textos_tickets = []
  direcciones = []
  direccion = lineas[3] if len(lineas) > 3 else None
  direcciones.append(direccion)

  precio_match = re.search(r'TOTAL.*\s\d{1,2},\d{1,2}', '\n'.join(lineas))
  precio_total = precio_match.group() if precio_match else None
  total_match = re.search(r'\d{1,2},\d{1,2}', precio_total)
  total_pagado = total_match.group() if total_match else None

  coincidencia_hora = re.search(r'(\d{2}:\d{2}:\d{2})', '\n'.join(lineas), flags=re.IGNORECASE)
  horas_entrada = coincidencia_hora.groups() if coincidencia_hora else None

  descuento_gen_matches = re.search(r'Euros conseguidos:\d{1,2},\d{1,2}', '\n'.join(lineas))
  descuento_gen = descuento_gen_matches.group() if descuento_gen_matches else None
  descuento_cifra_match = re.search(r'\d{1,2},\d{1,2}', descuento_gen)
  descuento_cifra = descuento_cifra_match.group() if descuento_cifra_match else None

  descuento_ac_matches = re.search(r'Saldo próximo Vale Mensual:\d{1,2},\d{1,2}', '\n'.join(lineas))
  descuento_ac = descuento_ac_matches.group() if descuento_ac_matches else None
  descuento_acum_match = re.search(r'\d{1,2},\d{1,2}', descuento_ac)
  descuento_acum = descuento_acum_match.group() if descuento_acum_match else None

  df = pd.DataFrame({
    'id ticket': id_ticket,
    'direccion': direcciones,
    'Hora': horas_entrada,
    'total precio': total_pagado,
    'Descuento Generado': descuento_cifra,
    'Descuento Acumulado': descuento_acum
  },index=[0])

  return df

In [49]:
ejercicio2_descuento('/content/Ticket-13_11_2023-2.pdf')

,id ticket,direccion,Hora,total precio,Descuento Generado,Descuento Acumulado
0,0195-02-23176767,"C/ Muñoz Degraíin, 5, Oviedo",19:57:52,"3,39","0,03","0,37"


## Función final
Finalmente, ponemos todas las partes anteriores en una misma funcion que nos devuelva directamente los csv que nos pide el enunciado.

In [108]:
def ejercicio2_final(tickets):
  pdf_reader = PyPDF2.PdfReader(tickets)
  ticket = pdf_reader.pages[0]
  texto_ticket = ticket.extract_text()
  lineas = np.array(texto_ticket.split('\n'))

  id_ticket_match = re.search(r'\d*-\d*-\d*', '\n'.join(lineas))
  id_ticket = id_ticket_match.group() if id_ticket_match else None

  productos = []
  for linea in lineas:
        producto_match = re.search(r'[A-ZÑñÁÉÍÓÚáéíóú][A-ZÑñÁÉÍÓÚáéíóú/0-9\s]+((\d{1,2},\d{1,2}\s*){1,2})', linea, flags=re.IGNORECASE)
        if producto_match:
            productos.append(producto_match.group())

  nombres = []
  for linea in productos:
    nombres_matches = re.search(r'[A-ZÑñÁÉÍÓÚáéíóú/0-9\s]+[A-ZÑñÁÉÍÓÚáéíóú]+\s', linea, flags=re.IGNORECASE)
    nombres.append(nombres_matches.group().strip() if nombres_matches else None)

  precios = []
  for linea in productos:
    precio_matches = re.search(r'(\d{1,2},\d{1,2})', linea)
    precios.append(precio_matches.group(1) if precio_matches else None)

  textos_tickets = []
  direcciones = []
  direccion = lineas[3] if len(lineas) > 3 else None
  direcciones.append(direccion)

  precio_match = re.search(r'TOTAL.*\s\d{1,2},\d{1,2}', '\n'.join(lineas))
  precio_total = precio_match.group() if precio_match else None
  total_match = re.search(r'\d{1,2},\d{1,2}', precio_total)
  total_pagado = total_match.group() if total_match else None

  coincidencia_hora = re.search(r'(\d{2}:\d{2}:\d{2})', '\n'.join(lineas), flags=re.IGNORECASE)
  horas_entrada = coincidencia_hora.groups() if coincidencia_hora else None

  descuento_gen_matches = re.search(r'Euros conseguidos:\d{1,2},\d{1,2}', '\n'.join(lineas))
  descuento_gen = descuento_gen_matches.group() if descuento_gen_matches else None
  descuento_cifra_match = re.search(r'\d{1,2},\d{1,2}', descuento_gen)
  descuento_cifra = descuento_cifra_match.group() if descuento_cifra_match else None

  descuento_ac_matches = re.search(r'Saldo próximo Vale Mensual:\d{1,2},\d{1,2}', '\n'.join(lineas))
  descuento_ac = descuento_ac_matches.group() if descuento_ac_matches else None
  descuento_acum_match = re.search(r'\d{1,2},\d{1,2}', descuento_ac)
  descuento_acum = descuento_acum_match.group() if descuento_acum_match else None

  df1 = pd.DataFrame({
    'id ticket': id_ticket,
    'direccion': direcciones,
    'Hora': horas_entrada,
    'total precio': total_pagado,
    'Descuento Generado': descuento_cifra,
    'Descuento Acumulado': descuento_acum
  },index=[0])

  df2 = pd.DataFrame({
    'id ticket': id_ticket,
    'producto': nombres,
    'precio_individual': precios
  })
  df2 = df2.dropna()
  df2 = df2.loc[df2['producto'] != 'TOTAL']
  df2 = df2.loc[df2['producto'] != 'TARJETA']

  df1_csv = df1.to_csv('df1_csv', index= False)
  df2_csv = df2.to_csv('df2_csv', index= False)


  return df1_csv, df2_csv

In [98]:
ejercicio2_final('/content/Ticket-13_11_2023-2.pdf')

(          id ticket                      direccion      Hora total precio  \
 0  0195-02-23176767  C/ Muñoz Degraíin, 5, Oviedo   19:57:52         3,39   
 
   Descuento Generado Descuento Acumulado  
 0               0,03                0,37  ,
           id ticket             producto precio_individual
 0  0195-02-23176767  ENSALADA RUCULA IFA              0,72
 1  0195-02-23176767               BAKERY              2,67)

In [99]:
ejercicio2_final('/content/Ticket-23_11_2023-2.pdf')

(          id ticket                      direccion      Hora total precio  \
 0  0195-02-23182951  C/ Muñoz Degraíin, 5, Oviedo   14:12:57         5,35   
 
   Descuento Generado Descuento Acumulado  
 0               0,05                0,62  ,
           id ticket              producto precio_individual
 0  0195-02-23182951  SOLOMILLO POLLO POZO              5,35)

In [107]:
ejercicio2_final('/content/Ticket-3_11_2023-2.pdf')

(          id ticket                      direccion      Hora total precio  \
 0  0195-02-23171045  C/ Muñoz Degraíin, 5, Oviedo   20:57:12         1,95   
 
   Descuento Generado Descuento Acumulado  
 0               0,01                0,04  ,
           id ticket            producto precio_individual
 0  0195-02-23171045  EUMMUS ENS2ALANDIA              1,25
 1  0195-02-23171045    PICOS IFA CAMPER              0,70)